In [1]:
import numpy as np

import os
from pathlib import Path

In [2]:
path = Path(os.getcwd())
root = Path(path.parent.parent.absolute())

preprocessed_image_path = root / 'Shared Preprocessed Objects' / 'Preprocessed Images for Inception' 
model_path = root / 'Models' / 'Retrained Inception'

In [3]:
cui2Index = np.load(model_path / "cui2Index.npy", allow_pickle=True).item()
training_pairs = np.load(model_path / "training_pairs.npy", allow_pickle=True).item()

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model

def get_untrained_inceptionV3(vocab_size):
    ## Initializing Pre-trained Model
    inceptionInput = keras.Input(shape=(299,299,3), name="img")
    model = InceptionV3(weights=None, input_tensor = inceptionInput)
    #model.summary()
    ## Removing Softmax, as the problem is not about classification
    model = Model(model.input, model.layers[-2].output)
    
    denseInput = keras.Input(shape=(2048,), name="denseInput")
    dense = tf.keras.layers.Dense(vocab_size, activation = 'sigmoid')(denseInput)#(model2.layers[-1].output)
    model_dense = Model(denseInput, outputs=dense)


    inceptionOut = model(inceptionInput)
    denseOut = model_dense(inceptionOut)

    model_final = Model(inputs = inceptionInput, outputs = denseOut)
    model_final.compile(optimizer='adam',loss='binary_crossentropy')#tf.keras.losses.BinaryCrossentropy(from_logits=False))
    model_final.summary()
    
    return model_final

In [5]:
vocab_size = len(cui2Index)
untrained_inceptionV3 = get_untrained_inceptionV3(vocab_size)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 299, 299, 3)]     0         
_________________________________________________________________
model (Functional)           (None, 2048)              21802784  
_________________________________________________________________
model_1 (Functional)         (None, 745)               1526505   
Total params: 23,329,289
Trainable params: 23,294,857
Non-trainable params: 34,432
_________________________________________________________________


In [6]:
import glob
import time
import gc

def get_x_y_from_batch(images, cuis):
    x, y = [], []
    for imageId in images:
        x.append(images[imageId])
        y.append(cuis[imageId])
    
    return np.array(x), np.array(y)

def get_cui_matrix_from_training_pairs(images, training_pairs, vocab_size):
    cuis_matrix = {}

    for index, img in enumerate(images):
        tmp = np.zeros(vocab_size)
        
        for value in training_pairs[img]:
            tmp[value] = 1
        
        cuis_matrix[img] = tmp
    
    return cuis_matrix

def training_loop(model, path, model_path, training_pairs, vocab_size, EPOCHS = 10, batch_size = 8):
    preprocessed_paths = glob.glob(str(path) + '/*')
    '''for index in range(len(preprocessed_paths)):
        preprocessed_paths[index] = preprocessed_paths[index].replace('\\', '//')'''
    #print(preprocessed_paths)

    history = []
    
    for epoch in range(EPOCHS):
        print('---- EPOCH ' + str(epoch) + ' ----')
        for count, path in enumerate(preprocessed_paths):
            start = time.time()
            if "test" in path:
                continue
            else:
                images = np.load(path, allow_pickle = True).item()
                
                cuis = get_cui_matrix_from_training_pairs(images, training_pairs, vocab_size)
                gc.collect()
                x, y = get_x_y_from_batch(images, cuis)
                gc.collect()
            
                #print(x.shape, y.shape)
                
                history.append(model.fit(x, y, batch_size = batch_size, verbose=1, validation_split = 0.05))
                tf.keras.backend.clear_session()
              
            print("Total Time for Batch = " + str(time.time() - start) + " seconds")
        model.save(model_path / 'Retrained_Inception' / ('epoch=' + str(epoch)))
    return model, np.array(history)

In [7]:
model, history = training_loop(untrained_inceptionV3, preprocessed_image_path, model_path, training_pairs, vocab_size)
np.save(model_path / 'history', history)

---- EPOCH 0 ----
2/2 [==============================] - 28s 7s/step - loss: 0.6942 - val_loss: 0.6842
Total Time for Batch = 29.693770170211792 seconds
2/2 [==============================] - 3s 732ms/step - loss: 0.2912 - val_loss: 0.6476
Total Time for Batch = 4.064781427383423 seconds
2/2 [==============================] - 3s 798ms/step - loss: 0.0905 - val_loss: 0.5868
Total Time for Batch = 3.794854164123535 seconds
2/2 [==============================] - 4s 829ms/step - loss: 0.0601 - val_loss: 0.5174
Total Time for Batch = 4.4703240394592285 seconds
2/2 [==============================] - 3s 790ms/step - loss: 0.0512 - val_loss: 0.4443
Total Time for Batch = 3.548741340637207 seconds
2/2 [==============================] - 3s 748ms/step - loss: 0.0725 - val_loss: 0.3403
Total Time for Batch = 3.88904070854187 seconds
2/2 [==============================] - 3s 713ms/step - loss: 0.0837 - val_loss: 0.2541
Total Time for Batch = 3.6330909729003906 seconds
2/2 [=========================

KeyboardInterrupt: 